  # task_One
  ## Analyzing output from infoTracker
  _Date: 03/27/2017_

In [2]:
import pandas as pd

In [118]:
df = pd.read_csv('/home/savy/Documents/2017_master_internship/infoTracker/prot_est_Infos.csv')

In [119]:
df.head()                                       

,Species name,Order name,Family name,#Protein,#EST
0,Catajapyx aquilonaris,Diplura,Japygidae,NaN,0
1,Orchesella cincta,Collembola,Entomobryidae,NaN,1146
2,Zootermopsis nevadensis,Blattodea,Termopsidae,14610.0,1
3,Nasutitermes exitiosus,Blattodea,Termitidae,NaN,0
4,Blattella germanica,Blattodea,Ectobiidae,NaN,8351


In [121]:
g = df.groupby(['Order name'])

In [131]:
g['Species name','#Protein','#EST'].max()         

,Species name,#Protein,#EST
Order name,,,
Blattodea,Zootermopsis nevadensis,14610.0,8351
Coleoptera,Tribolium castaneum,18534.0,185437
Collembola,Orchesella cincta,NaN,1146
Diplura,Catajapyx aquilonaris,NaN,0
Diptera,Zaprionus indianus,23304.0,847942
Ephemeroptera,Ephemera danica,NaN,0
Hemiptera,Trionymus perrisii,NaN,219173
Hymenoptera,Wasmannia auropunctata,18742.0,174597
Lepidoptera,Spodoptera frugiperda,16912.0,575578


## Download fasta file from NCBI

In [133]:
from Bio import Entrez
Entrez.email = 'savandara.besse@gmail.com'

In [145]:
handle = Entrez.esearch(db="nucest", term='drosophila melanogaster')
record = Entrez.read(handle)['IdList']
handle.close()
record

['527482192', '392859066', '345421511', '342206684', '315905243', '315844150', '315823897', '315823706', '315820860', '315809918', '315803644', '315802413', '315798677', '311205018', '311147959', '302181891', '297251322', '297251281', '297251268', '297251209']

In [151]:
import os
from Bio import SeqIO
from Bio import Entrez
net_handle = Entrez.efetch(db="nucest", id='527482192', rettype="fasta", retmode="text")
out_handle = open('test.fna', "w")
out_handle.write(net_handle.read())
out_handle.close()
net_handle.close()
print("Saved")

Saved


In [152]:
print("Parsing...")
record = SeqIO.read('test.fna', "fasta")
print(record)

Parsing...
ID: JZ478789.1
Name: JZ478789.1
Description: JZ478789.1 01RQ1.12 WSSP Duckweed cDNA library Landoltia punctata cDNA clone 01RQ1.12 5' similar to protein translocation protein, mRNA sequence
Number of features: 0
Seq('AGAGAGATCGTAGAGGGATTCGGTCAAGAGTACGTTCCCCAGAGGGTTCTACCG...GCA', SingleLetterAlphabet())


In [168]:
search_handle = Entrez.esearch(db="nucest",term="drosophila melanogaster", usehistory="y")
search_results = Entrez.read(search_handle)
search_handle.close()

In [169]:
search_results

{'QueryKey': '1', 'TranslationSet': [{'To': '"Collembola"[Organism] OR "Collembola"[Organism] OR collembola[All Fields]', 'From': 'collembola'}], 'IdList': ['429477244', '429477243', '289812815', '284004874', '284004873', '284004872', '284004871', '284004870', '284004869', '284004868', '284004867', '284004866', '284004865', '284004864', '284004863', '284004862', '284004861', '284004860', '284004859', '284004858'], 'WebEnv': 'NCID_1_176873589_130.14.22.215_9001_1490652757_2105856053_0MetA0_S_MegaStore_F_1', 'Count': '34031', 'QueryTranslation': '"Collembola"[Organism] OR "Collembola"[Organism] OR collembola[All Fields]', 'RetStart': '0', 'TranslationStack': [{'Count': '34031', 'Explode': 'Y', 'Field': 'Organism', 'Term': '"Collembola"[Organism]'}, {'Count': '34031', 'Explode': 'Y', 'Field': 'Organism', 'Term': '"Collembola"[Organism]'}, 'OR', {'Count': '16', 'Explode': 'N', 'Field': 'All Fields', 'Term': 'collembola[All Fields]'}, 'OR', 'GROUP'], 'RetMax': '20'}

In [173]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]
gi_list = search_results["IdList"]
count = int(20)

In [174]:
try:
    from urllib.error import HTTPError  # for Python 3
except ImportError:
    from urllib2 import HTTPError  # for Python 2

In [175]:
batch_size = 1
out_handle = open("orchid_rpl16.fasta", "w")
for start in range(0, count, batch_size):
    end = min(count, start+batch_size)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    while attempt < 3:
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="nucest",
                                         rettype="fasta", retmode="text",
                                         retstart=start, retmax=batch_size,
                                         webenv=webenv, query_key=query_key)
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    data = fetch_handle.read()
    fetch_handle.close()
    out_handle.write(data)
out_handle.close()